In [1]:
1+1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from werkzeug.datastructures import FileStorage
import pandas as pd
import os, tempfile

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
print("-----reading data from azure")
storage_account_name = "pocmosaic"
storage_account_access_key = "iX6jZejeNh1bUejdSRYk+4X2S/pAbTy+idkTM5UAw2HxgxSnaYGJSN+ffzZAQ5/Q6xQe2Ja74Re9nKHMVmsGhA=="
file_location = "wasbs://container1@pocmosaic.blob.core.windows.net/catalog/local_file_upload/RatanBBostonHousing.csv"
file_type = "csv"
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
dataset = spark.read.option("header", "true").format(file_type).option("inferSchema", "true").load(file_location)
dataset.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----reading data from azure
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631

In [4]:
print("-----splitting data")
train_data,test_data = dataset.randomSplit([0.8,0.2])

print("-----creating pipeline")
stage_1 = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
# define stage 5: logistic regression model                          
stage_2 = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

# setup the pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----splitting data
-----creating pipeline

In [5]:
print("-----training pipeline")
regression_pipeline = regression_pipeline.fit(train_data)
test_update = regression_pipeline.transform(test_data)
# test_update.show()

#Predict using the model
print("-----test data")
test_update.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----training pipeline
-----test data
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|          Attributes|        prediction|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
| 0.0136|75.0|  4.0|   0|  0.41|5.888|47.6|7.3197|  3|469|   21.1| 396.9| 14.8|18.9|[0.0136,75.0,4.0,...|  15.4083418235927|
| 0.0187|85.0| 4.15|   0| 0.429|6.516|27.7|8.5353|  4|351|   17.9|392.43| 6.36|23.1|[0.0187,85.0,4.15...|25.222978133139154|
|0.01951|17.5| 1.38|   0|0.4161|7.104|59.5|9.2229|  3|216|   18.6|393.24| 8.05|33.0|[0.01951,17.5,1.3...|23.156016471958235|
|0.02009|95.0| 2.68|   0|0.4161|8.034|31.9| 5.118|  4|224|   14.7|390.55| 2.88|50.0|[0.02009,95.0,2.6...| 42.16749722576602|
|0.02177|82.5| 2.03|   0| 0.415| 7.61|15.7|  6.27|  2|348|   14.7|395.38| 3.11|42.3|[0.

In [7]:
# define score function
def score(model, request):
    data = request.json
    df = pd.DataFrame(data, columns=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv'])
    spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
    spark_df = spark.createDataFrame(df)
    return model.transform(spark_df).toPandas()["prediction"].to_json()

print("-----defining & testing score function")
import requests
req = requests.Request()
req.json = [[1, -0.1, 0.0425, 0.0279, 2.4716, -15.2808, 4.2783, -0.0005, -1.4009, 0.2692, -0.013, -0.9905, 0.0084, -0.5533]]
rez = score(regression_pipeline, req)
print("the prediction is {}".format(rez))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----defining & testing score function
the prediction is {"0":-56.8627787775}

In [8]:
%%local
import os
ai_server = os.environ["MOSAIC_AI_SERVER"]
project_id = os.environ["PROJECT_ID"]
token = os.environ["TOKEN"]


In [9]:
%%send_to_spark -i ai_server -t str -n ai_server


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'ai_server' as 'ai_server' to Spark kernel

In [10]:
%%send_to_spark -i project_id -t str -n project_id


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'project_id' as 'project_id' to Spark kernel

In [11]:
%%send_to_spark -i token -t str -n token


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'token' as 'token' to Spark kernel

In [12]:
import os
os.environ["MOSAIC_AI_SERVER"] = ai_server
os.environ["PROJECT_ID"] = project_id
os.environ["TOKEN"] = token


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from mosaicml import *
from mosaicml import constants
constants.MosaicAI.server = "http://mosaic-ai-backend:5000/registry/api"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
register_model(regression_pipeline, score, "Boston_Regression_Analysis_inter_july5", "pyspark model", MLModelFlavours.pyspark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…